In [21]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("KNN")
sc = SparkContext.getOrCreate()

# getting the data in a (y, (x1,...xn)) format where y is the class label 
# and x1,...,xn are the predictive attribute values
# mydata = sc.textFile().split().map(lambda x: (x[0], (x[1:])))

# example of mydata
mydata = ((0,(2,1)), (1,(4,5)), (0,(1,3)), (0,(-2,1)), (1,(5,3)), (0,(1,1)), (1, (2,2.5)), (1, (3,5)))
# we use parallelize to partition the tuple so spark can work on it in parallel
data_rdd = sc.parallelize(mydata)


# number of folds
K = 2
fold_size = int(len(mydata) / K)

# ((y, (x1, x2)), fold id)
CVdata = tuple((mydata[k:k + fold_size],k) for k in range(0, len(mydata), fold_size))
CVdata = sc.parallelize(CVdata)

error = sc.emptyRDD()
p = (2,3) # point to classify
for i in range(0, len(mydata), fold_size):
    # divide data to train and the complement to test
    train = CVdata.filter(lambda x: x[1] != i).flatMap(lambda x: x[0])
    test  = CVdata.filter(lambda x: x[1] == i).flatMap(lambda x: x[0])
    # (error, 1)
    testError = test.map(lambda x: (x[0][0] - MWC(p,i,h))**2, 1)
    # add all errors together as well as counts
    testError = testError.reduce(lambda a,b: (a[0]+b[0], a[1]+b[1]))
    # get the average error of each iteration
    error[i] = x[0]/x[1]

# get the average test error of the whole cross validation
MSE = mean(error)

print(f"Average Test Errors: {MSE}")

train data: [(0, (2, 1)), (1, (4, 5)), (0, (1, 3)), (0, (-2, 1))]
test data: [(1, (5, 3)), (0, (1, 1)), (1, (2, 2.5)), (1, (3, 5))]


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 22.0 failed 1 times, most recent failure: Lost task 3.0 in stage 22.0 (TID 91, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 377, in main
  File "C:\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 372, in process
  File "C:\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark-2.4.5-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\util.py", line 99, in wrapper
    return f(*args, **kwargs)
  File "<ipython-input-21-1a7528c76d4c>", line 28, in <lambda>
NameError: name 'LR' is not defined

	at org.apache.spark.api.python.Base